# Deployment Insights ETL and SQL

ETL for the Deployment Insights database with sample SQL queries using Trino and the Aerospike Trino Connector.

### Ensure that the Aerospike Database is running

In [67]:
!asd >& /dev/null
!pgrep -x asd >/dev/null && echo "Aerospike database is running!" || echo "**Aerospike database is not running!**"

Aerospike database is running!


### Initialize Client
Initialize Python Client used to access features stored in the Aerospike feature store.

In [77]:
import aerospike
import sys
# connect to the database
config = {
  'hosts': [ ('127.0.0.1', 3000) ]
}
try:
  client = aerospike.client(config).connect('pensive','p3n$1v3')
except:
  print("failed to connect to the cluster with", config['hosts'])
  sys.exit(1)
print('Client initialized and connected to database')

Client initialized and connected to database


# Extract Insights from Collect-Info Documents
A Support case has attachments that are produced by the collect-info tool. The tgz archive of the attachments consists of many files, of which we use the following to extract insights:
- [timestamp]_ascinfo.json
- [timestamp]_ascollectinfo.log

In the future we will also use this source from the archive:
- [timestamp]_summary.log

Extract the case tgz archives that you want to process, and upload the above files into a directory. Set `DATA_DIR` to this directory below. The notebook uses one json file as an example to illistrate the ETL process in the first part. Pick one json file and assign it to `EXAMPLE_JSON_FILE`.

In the later part, all the files in the `DATA_DIR` directory are batch processed.

In [33]:
DATA_DIR = '../data'
EXAMPLE_JSON_FILE = '20220115_004012_ascinfo.json'

In [34]:
import json

def readJsonDataFromFile(jsonFile):  
    # JSON file
    f = open (jsonFile, "r")
    # Reading from file
    data = json.loads(f.read())
    # Closing file
    f.close()
    return data

json_file =  DATA_DIR + '/' + EXAMPLE_JSON_FILE    #example json file path
ascinfo_json = readJsonDataFromFile(json_file)
print('Read ascinfo json file:', json_file)

Read ascinfo json file: ../data/20220115_004012_ascinfo.json


In [35]:
# extract top level info: timestamp and cluster
def get_timestamp(json):
    timestamp = list(json.keys())[0]
    return timestamp

def get_cluster_items(json):
    ts_items = list(json.items())
    cluster_items = list(ts_items[0][1].items())
    return cluster_items

def get_node_items(cluster_items):
    return list(cluster_items[0][1].items())

timestamp = get_timestamp(ascinfo_json)
print('timestamp:', timestamp)

cluster_itmes = get_cluster_items(ascinfo_json)
cluster_name = cluster_itmes[0][0]
print('cluster name:', cluster_name)

node_items = get_node_items(cluster_itmes)
num_nodes = len(node_items)
print('number of nodes:', num_nodes)

timestamp: 2022-01-15 00:40:12 UTC
cluster name: conv-euw1
number of nodes: 6


# Collect Insights
Collect the following cluster insights:

- name
- timestamp
- num_nodes 
- num_ns
- features: List (xdr, strong_consistency, single_bin, data_in_index, ...)
- num_objects
- storage_engines
- num_device_bytes
- num_memory bytes

Collect the following namespace insights:
- name
- strong_consistency
- num_secondary_indices
- num_sets
- num_bins
- num_device_bytes
- num_memory_bytes
- num_objects
- replication_factor
- single_bin (true/false)
- data_in_index (true/false)
- storage_engine


### Namespace Info

Collect features for each distinct namespace from each node and aggregate stats (device/mem_bytes, objects, storage_engines):
```
for each node:
    for each namespace:
        if new, 
            add namespace to namespaces map
            copy features
        # else: ensure settings are same
        Aggregate stats      
```

In [36]:
def get_namespace_info(node_items):
    # Potential future additions:
                #Secondary index #, types?
                #Total device bytes + total memory bytes - aggregate from sets    namespace_info = {}
    namespace_info = {}
    for node, node_subtree in node_items:
        for ns, ns_subtree in node_subtree['as_stat']['statistics']['namespace'].items():
            if ns not in namespace_info:
                nsinfo = {}
                nsinfo['name'] = ns
                nsinfo['num_bins'] = ns_subtree['bin'].get('bin_names', 0) 
                nsinfo['num_sets'] = len(ns_subtree['set'])
                nsinfo['num_sindex'] = len(ns_subtree['sindex'])
                nsinfo['replication_factor'] = ns_subtree['service'].get('replication-factor', 0) 
                nsinfo['storage_engine'] = ns_subtree['service']['storage-engine']
                nsinfo['ns_cluster_size'] = ns_subtree['service'].get('ns_cluster_size', 0)
                nsinfo['master_objects'] = ns_subtree['service']['master_objects']
                nsinfo['objects'] = ns_subtree['service']['objects']
                nsinfo['single_bin'] = ns_subtree['service']['single-bin']
                nsinfo['strong_consistency'] = ns_subtree['service'].get('strong-consistency', 'false')
                nsinfo['data_in_index'] = ns_subtree['service']['data-in-index']
                namespace_info[ns] = nsinfo
    return namespace_info

namespace_info = get_namespace_info(node_items)
print(namespace_info)

{'memory': {'name': 'memory', 'num_bins': '5', 'num_sets': 2, 'num_sindex': 0, 'replication_factor': '3', 'storage_engine': 'memory', 'ns_cluster_size': '6', 'master_objects': '226', 'objects': '604', 'single_bin': 'false', 'strong_consistency': 'false', 'data_in_index': 'false'}}


### Cluster Info

Collect features at the cluster level:

- Case, customer, timestamp, num_nodes
- edition, asd-build
- Aggregated from namespaces: num_ns, features, device/mem bytes, objects, storage engines.


In [37]:
def get_cluster_info(timestamp, cluster_name, node_items, namespace_info):
    cluster_info = {}
    cluster_info['timestamp'] = timestamp
    cluster_info['cluster_name'] = cluster_name 
    cluster_info['cluster_size'] = len(node_items)
    #cluster_info['edition'] = node_items[0][1]['as_stat']['meta_data'].get('edition', 'unspecified')
    cluster_info['server_release'] = node_items[0][1]['as_stat']['meta_data']['asd_build']
    #cluster_info['num_namespaces'] = len(namespace_info)

    cluster_info['total_objects'] = 0
    cluster_info['storage_engines'] = set()
    for nsinfo in namespace_info.values():
        cluster_info['total_objects'] = cluster_info['total_objects'] + int(nsinfo['objects']) 
        cluster_info['storage_engines'] |= set([nsinfo['storage_engine']]) 
        #cluster_info['features_in_use'] |= set(['single_bin'] if nsinfo['single_bin'] == 'true' else [])
        #cluster_info['features_in_use'] |= set(['strong_consistency'] if nsinfo['strong_consistency'] == 'true' else [])
        #cluster_info['features_in_use'] |= set(['data_in_index'] if nsinfo['data_in_index'] == 'true' else [])
    cluster_info['storage_engines'] = list(cluster_info['storage_engines'])
    return cluster_info

FEATURE_KEY_MAP = {
    'AGGREGATION' : 'aggregation',
    'BATCH' : 'batch',
    'INDEX-ON-DEVICE' : 'index_on_device',
    'INDEX-ON-PMEM' : 'index_on_pmem',
    'KVS' : 'kvs',
    'LDT' : 'ldt',
    'QUERY' : 'query',
    'RACK-AWARE' : 'rack_aware',
    'SC' : 'sc',
    'SCAN' : 'scan',
    'SECURITY' : 'security',
    'SINDEX' : 'sindex',
    'TLS (FABRIC)' : 'tls_fabric',
    'TLS (HEARTBEAT)' : 'tls_heartbeat',
    'TLS (SERVICE)' : 'tls_service',
    'UDF' : 'udf',
    'XDR DESTINATION' : 'xdr_dest',
    'XDR SOURCE' : 'xdr_src'
}

def set_features_in_use(json_path, cluster_info):
    '''
    for node, node_subtree in node_items:
        cluster_info['features_in_use'] |= set(['xdr'] if len(node_subtree['as_stat']['config'].get('xdr', {})) > 0 else [])
        # infer other features
    cluster_info['features_in_use'] = list(cluster_info['features_in_use'])
    '''
    log_file = json_path[:-12] + 'ascollectinfo.log'
    print ('ascollectinfo log file:', log_file)
    # sed -n '/Features/,/ASCOLLECTINFO/p' 20220115_004012_ascollectinfo.log | grep YES | sed 's/[ \t]*:.*//'
    import os
    stream = os.popen("sed -n '/Features/,/ASCOLLECTINFO/p' " + log_file +  " | grep ' YES' | sed 's/[ \t]*:.*//'")
    features = stream.read().strip().split('\n')
    print('features read:', features)
    cluster_info['features'] = set()  
    for feature in features:
        if feature not in FEATURE_KEY_MAP:
            continue
        cluster_info['features'] |= set([FEATURE_KEY_MAP[feature]])
    cluster_info['features'] = list(cluster_info['features'])
    return cluster_info
    
cluster_info = get_cluster_info(timestamp, cluster_name, node_items, namespace_info)
cluster_info = set_features_in_use(json_file, cluster_info)
print('cluster info:', cluster_info)


ascollectinfo log file: ../data/20220115_004012_ascollectinfo.log
features read: ['KVS', 'TLS (FABRIC)', 'TLS (SERVICE)']
cluster info: {'timestamp': '2022-01-15 00:40:12 UTC', 'cluster_name': 'conv-euw1', 'cluster_size': 6, 'server_release': '5.5.0.9', 'total_objects': 604, 'storage_engines': ['memory'], 'features': ['kvs', 'tls_service', 'tls_fabric']}


# Load Into Database
Insert the case record with case-number as the (user) key, and bins: timestamp, customer, cluster, and namespaces.

In [38]:
NAMESPACE = 'test'
SET = 'insights'
def load(case_num, customer, timestamp, cluster_info, namespace_info):
    client.put((NAMESPACE, SET, case_num), 
               {'case_num': case_num,
                'customer': customer,
                **cluster_info,
               'namespaces':list(namespace_info.values())})
    return

CASE_NUM = 100
CUST_NAME = 'Widgets, Inc.'
load(CASE_NUM, CUST_NAME, timestamp, cluster_info, namespace_info)
print('record inserted into the database')

record inserted into the database


## Validate Data in Database

In [39]:
!aql -c "set output raw; select * from test.insights"

set output raw
OUTPUT = RAW
select * from test.insights
*************************** 1. row ***************************
case_num: 100
customer: "Widgets, Inc."
timestamp: "2022-01-15 00:40:12 UTC"
cluster_name: "conv-euw1"
cluster_size: 6
server_release: "5.5.0.9"
total_objects: 604
storage_engines: LIST('["memory"]')
features: LIST('["kvs", "tls_service", "tls_fabric"]')
namespaces: LIST('[{"ns_cluster_size":"6", "replication_factor":"3", "num_bins":"5", "storage_engine":"memory", "num_sets":2, "name":"memory", "data_in_index":"false", "single_bin":"false", "objects":"604", "master_objects":"226", "num_sindex":0, "strong_consistency":"false"}]')

1 row in set (0.246 secs)

OK




# Batch ETL: Process All Files
Now we ETL all ascinfo.json and ascollectinfo.log files in the `DATA_DIR` directory.

We assign fictious case numbers and customer names for now to the data (later to be obtained from the Support database). We sequentially assign case numbers starting at `CASE_NUM`, and rotate customer names from the `CUST_NAMES` array.

In [40]:
CASE_NUM_START = 200
CUST_NAMES = ['Widgets, Inc','Wares Corp','Parts Ltd','Component Factory','Modular Design','We Assemble']

In [41]:
import glob
def process_batch_etl():
    case_num = CASE_NUM_START
    for json_file in glob.glob(DATA_DIR + '/*_ascinfo.json'):
        json_info = readJsonDataFromFile(json_file)
        print('Read ascinfo json file:', json_file)
        
        timestamp = get_timestamp(json_info)
        print('timestamp:', timestamp)

        cluster_itmes = get_cluster_items(json_info)
        cluster_name = cluster_itmes[0][0]
        print('cluster name:', cluster_name)

        node_items = get_node_items(cluster_itmes)
        num_nodes = len(node_items)
        print('number of nodes:', num_nodes)

        namespace_info = get_namespace_info(node_items)
        print(namespace_info)
        
        cluster_info = get_cluster_info(timestamp, cluster_name, node_items, namespace_info)
        cluster_info = set_features_in_use(json_file, cluster_info)
        print(cluster_info)

        cust_name = CUST_NAMES[case_num % len(CUST_NAMES)]
        load(case_num, cust_name, timestamp, cluster_info, namespace_info)
        case_num += 1
        print('record inserted into the database')        
        
        print('\n')
    return
    
process_batch_etl()
print('batch etl done.')

Read ascinfo json file: ../data/20190315_203751_ascinfo.json
timestamp: 2019-03-15 20:39:59 UTC
cluster name: null
number of nodes: 34
{'users': {'name': 'users', 'num_bins': '9', 'num_sets': 2, 'num_sindex': 0, 'replication_factor': 0, 'storage_engine': 'device', 'ns_cluster_size': 0, 'master_objects': '1377143747', 'objects': '2361400096', 'single_bin': 'false', 'strong_consistency': 'false', 'data_in_index': 'false'}, 'context': {'name': 'context', 'num_bins': '14', 'num_sets': 1, 'num_sindex': 0, 'replication_factor': 0, 'storage_engine': 'device', 'ns_cluster_size': 0, 'master_objects': '111906126', 'objects': '221789094', 'single_bin': 'false', 'strong_consistency': 'false', 'data_in_index': 'false'}}
ascollectinfo log file: ../data/20190315_203751_ascollectinfo.log
features read: ['BATCH', 'KVS', 'SCAN']
{'timestamp': '2019-03-15 20:39:59 UTC', 'cluster_name': 'null', 'cluster_size': 34, 'server_release': '3.13.0.11', 'total_objects': 2583189190, 'storage_engines': ['device'], '

## Examine Database

In [42]:
!aql -c "set output raw; select * from test.insights"

set output raw
OUTPUT = RAW
select * from test.insights
*************************** 1. row ***************************
case_num: 201
customer: "Component Factory"
timestamp: "2022-01-15 00:40:12 UTC"
cluster_name: "conv-euw1"
cluster_size: 6
server_release: "5.5.0.9"
total_objects: 604
storage_engines: LIST('["memory"]')
features: LIST('["kvs", "tls_service", "tls_fabric"]')
namespaces: LIST('[{"ns_cluster_size":"6", "replication_factor":"3", "num_bins":"5", "storage_engine":"memory", "num_sets":2, "name":"memory", "data_in_index":"false", "single_bin":"false", "objects":"604", "master_objects":"226", "num_sindex":0, "strong_consistency":"false"}]')
*************************** 2. row ***************************
case_num: 200
customer: "Parts Ltd"
timestamp: "2019-03-15 20:39:59 UTC"
cluster_name: "null"
cluster_size: 34
server_release: "3.13.0.11"
total_objects: 2583189190
storage_engines: LIST('["device"]')
features: LIST('["batch", "kvs", "scan"]')
namespaces: LIST('[{"ns_cluster_siz

# Trino SQL Queries
For the following queries to exexute in the notebook, you must have a Trino server running at port 8080 of the host, connected to this container's Aerospike database via the Aerospike Trino Connector.

## Trino Setup
1. Make sure the port of the Aerospike server (typically 3000) running in this notebook's container is exposed to host. 
2. [Use these instruactions](https://github.com/citrusleaf/aerospike-connect-trino#run-on-docker) to run Trino and the Aerospike Trino Connector in a docker container on the host machine. Make sure the Trino Connector can access the Aerospike server in this container.
3. [Install the Trino Client](https://trino.io/docs/current/installation/cli.html) in this container. The following cells assume it is installed in the parent directory.

## Trino Command
Define the environment variable for short form of the Trino command. 

You can also run the Trino command line tool in a separate shell tab.

In [2]:
%env TRINO=../trino --server host.docker.internal:8080 --catalog aerospike --schema test --output-format=TSV_HEADER
%env TRINO_VERTICAL=../trino --server host.docker.internal:8080 --catalog aerospike --schema test --output-format=VERTICAL

env: TRINO=../trino --server host.docker.internal:8080 --catalog aerospike --schema test --output-format=TSV_HEADER
env: TRINO_VERTICAL=../trino --server host.docker.internal:8080 --catalog aerospike --schema test --output-format=VERTICAL


## Examples

Show schemas (namespaces).

In [9]:
!$TRINO --execute "show schemas";

Schema
bar
information_schema
test


Show tables (sets).

In [12]:
!$TRINO --execute "show tables";

Table
__default
insights


Show a sample record.

In [13]:
!$TRINO_VERTICAL --execute "select * from test.insights limit 1" ;


-[ RECORD 1 ]---+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
__key           | NULL
case_num        | 25488
cluster_name    | userprofile-aerospike-us-central1-prod
cluster_size    | 9
customer        | Snapchat (Snap, Inc.)
features        | ["batch","scan","rack_aware","security","kvs"]
namespaces      | [{"num_sets":5,"replication_factor":"3","num_sindex":0,"data_in_index":"false","num_bins":"15","objects":"759044874","name":"persistent","single_bin":"false","ns_cluster_size":"9","storage_engine":"device","master_objects":"61328743","strong_consistency":"false"}]
server_release  | 5.2.0.4
storage_engines | ["device"]
timestamp       | 2021-11-12 17:45:59 UTC
total_objects   | 759044874


Get customers using feature 'index_on_device'.

Get customers using feature 'index_on_device'.
```
select customer, cluster_name, features 
from insights 
where contains(cast(features as array(VARCHAR)),'index_on_device')
```

In [14]:
!$TRINO --execute "select customer, cluster_name, features from insights where contains(cast(features as array(VARCHAR)),'index_on_device')" ;


customer	cluster_name	features
LexisNexis Risk Solutions Inc. (ThreatMetrix)	EASTUS2non	["udf","security","xdr_dest","xdr_src","batch","index_on_device","kvs"]
Charles Schwab	TIGER-QA1	["tls_service","query","index_on_device","kvs","tls_fabric","sindex","scan","batch","rack_aware"]
Charles Schwab	TIGER-PDC3	["tls_fabric","kvs","udf","sindex","index_on_device","batch","tls_service","rack_aware","xdr_dest"]


Get customers using feature 'xdr_dest' and release after 5.x
```
select customer, features, server_release 
from insights 
where contains(cast(features as array(VARCHAR)),'xdr_dest') and regexp_like(server_release, '^5.*')
```

In [16]:
!$TRINO --execute "select customer, features, server_release from insights where contains(cast(features as array(VARCHAR)),'xdr_dest') and regexp_like(server_release, '^5.*');" ;


customer	features	server_release
DoubleVerify	["udf","xdr_dest","kvs"]	5.2.0.6
Nokia Solutions and Networks OY (Channels Account)	["query","scan","tls_fabric","kvs","aggregation","sindex","tls_service","rack_aware","xdr_dest"]	5.6.0.4
Charles Schwab	["tls_fabric","kvs","udf","sindex","index_on_device","batch","tls_service","rack_aware","xdr_dest"]	5.2.0.21
DoubleVerify	["kvs","udf","xdr_dest","scan"]	5.2.0.6
Nokia Solutions and Networks OY (Channels Account)	["kvs","sindex","query","xdr_dest","aggregation","rack_aware"]	5.6.0.4


Get the largest deployed cluster by each customer.
```
select customer, max(cluster_size) as max_cluster_size
rom insights
group by customer
```

In [17]:
!$TRINO --execute "select customer, max(cluster_size) as max_cluster_size from insights group by customer" ;


customer	max_cluster_size
Barclays PLC	78
Adobe Systems Incorporated	12
VPBank SMBC Finance Company Limited	4
Passendo ApS	12
Express (Hangzhou) Technology Service Company Limited	3
Charles Schwab	9
Supermarket Grocery Supplies Pvt Ltd (BigBasket.com)	24
PingAn Bank	5
LexisNexis Risk Solutions Inc. (ThreatMetrix)	22
Deqwas, Inc.	14
Phonepe Private Limited (FX Mart Pvt. Ltd.)	10
Aerospike Cloud Managed Service	3
Federal Reserve Bank of Boston	3
Verizon Media	102
Clinch Labs Ltd	4
PubMatic	8
Cybereason Inc	14
DoubleVerify	12
Starkware Industries Ltd	31
Bombinate Technologies Pvt Ltd.	16
Affle International Pte Ltd	14
Sovrn Holdings	5
Quantcast Corporation	28
Ping An Bank	7
Amdocs Development Limited (Channels Account)	18
PayPal, Inc.	30
YouAppi INC	3
PT GoTo Gojek Tokopedia	12
AdTalos  - 新义互联科技有限公司	10
AppsFlyer	74
IDFC First Bank Ltd	3
Verizon Media (Dublin Ireland)	127
WalkMe LTD	15
Neustar, Inc. - San Diego	14
StackAdapt	5
ironSource Ltd	25
Wayfair LLC	36
Knorex Pte Ltd	2
PVR Limited	3

Get the largest namespace by objects for each customer.
```
select customer, max(transform(cast(namespaces as array<map<varchar, varchar>>), entry->entry['objects'])) 
                 as max_ns_objects
from insights 
group by customer
```

In [18]:
!$TRINO --execute "select customer, max(transform(cast(namespaces as array<map<varchar, varchar>>), entry->entry['objects'])) as max_ns_objects from insights group by customer" ;


customer	max_ns_objects
Express (Hangzhou) Technology Service Company Limited	[6785]
Charles Schwab	[81530124, 285080914, 6952935208, 67966191]
PingAn Bank	[0, 99596584, 2057268387, 116092797, 0]
Supermarket Grocery Supplies Pvt Ltd (BigBasket.com)	[145423600, 9863061]
Barclays PLC	[682387527]
Adobe Systems Incorporated	[680914382, 0]
VPBank SMBC Finance Company Limited	[23839637, 74399476]
Phonepe Private Limited (FX Mart Pvt. Ltd.)	[6835608, 121806857]
LexisNexis Risk Solutions Inc. (ThreatMetrix)	[9202388756]
Deqwas, Inc.	[857023949]
Aerospike Cloud Managed Service	[119, 111655]
Federal Reserve Bank of Boston	[0]
Passendo ApS	[68659919, 1353548]
Starkware Industries Ltd	[89491151]
Affle International Pte Ltd	[23973843, 0, 3630870, 16815037, 11905, 9694699, 372875238, 5981, 3710680]
Sovrn Holdings	[2121633662]
Quantcast Corporation	[29031, 571591753, 32006, 11773679, 1795895755, 573178]
Bombinate Technologies Pvt Ltd.	[2700172909]
YouAppi INC	[15642113, 10422573, 0]
PayPal, Inc.	[729

Get customers having namespaces configured with:
    storage-engine: data-in-memory AND 
    single-bin
    Include enable-xdr if present (dropped in later versions).
`
    select customer, transform(cast(namespaces as array<map<varchar, varchar>>), entry->(entry['single_bin'], entry['storage_engine'])) as single_bin_device 
    from insights
    where single_bin_device = {true, memory}
`

Get customers that have single-bin in-memory namespaces.

`
select customer, single_bin_mem_ns from (
    select customer,  
      filter(
        transform(cast(namespaces as array<map<varchar,varchar>>), 
                  ns->map_filter(ns,(k,v)->k in ('name','single_bin','storage_engine'))), 
        ns->ns['single_bin']='true' and ns['storage_engine'] = 'memory') 
        as single_bin_mem_ns 
from insights) 
where single_bin_mem_ns != Array[]";
`

In [23]:
!$TRINO --execute "select customer, single_bin_mem_ns from (select customer, filter(transform(cast(namespaces as array<map<varchar,varchar>>), ns->map_filter(ns,(k,v)->k in ('name','single_bin','storage_engine'))), ns->ns['single_bin']='true' and ns['storage_engine'] = 'memory') as single_bin_mem_ns from insights) where single_bin_mem_ns != Array[]";


customer	single_bin_mem_ns
Quantcast Corporation	[{name=in-memory-rtff, single_bin=true, storage_engine=memory}]


Get customers that have single-bin in-memory namespaces, with XDR flag to indicate use of XDR.

`
select customer, xdr, single_bin_mem_ns from (
    select customer, contains(cast(features as array<varchar>), 'xdr') as xdr, 
      filter(
        transform(cast(namespaces as array<map<varchar,varchar>>), 
                  ns->map_filter(ns,(k,v)->k in ('name','single_bin','storage_engine'))), 
        ns->ns['single_bin']='true' and ns['storage_engine'] = 'memory') 
        as single_bin_mem_ns 
from insights) 
where single_bin_mem_ns != Array[]";
`

In [24]:
!$TRINO --execute "select customer, xdr, single_bin_ns from (select customer, contains(cast(features as array<varchar>), 'xdr') as xdr, filter(transform(cast(namespaces as array<map<varchar,varchar>>), ns->map_filter(ns,(k,v)->k in ('name','single_bin','storage_engine'))), ns->ns['single_bin']='true' and ns['storage_engine'] = 'memory') as single_bin_ns from insights) where single_bin_ns != Array[]";


customer	xdr	single_bin_ns
Quantcast Corporation	false	[{name=in-memory-rtff, single_bin=true, storage_engine=memory}]


# Programmatic Access via Trino
Install the trino package if not already installed to enable Python access by running the following cell.

In [51]:
#!pip install trino


In [52]:
import trino
conn = trino.dbapi.connect(
    host='host.docker.internal',
    port=8080,
    user='admin',
    catalog='aerospike',
    schema='test'
)
cur = conn.cursor()
cur.execute('select * from insights limit 1')
rows = cur.fetchall()
print(rows)

[[None, 205, 'aerospike-clicks-pointers-036-eu1', 39, 'Wares Corp', '["kvs","scan","batch","security","index_on_device"]', '[{"num_sets":3,"replication_factor":"2","num_sindex":0,"data_in_index":"false","num_bins":0,"objects":"15512101854","name":"clicks-pointers","single_bin":"true","ns_cluster_size":"39","storage_engine":"device","master_objects":"8428991267","strong_consistency":"false"}]', '4.8.0.9', '["device"]', '2020-12-27 17:18:47 UTC', 15512101854]]


# Cleanup
Run the following cell to truncate the data.

In [53]:
!aql -c "truncate test.insights"

truncate test.insights
OK




In [75]:
client.close()